<a href="https://colab.research.google.com/github/yuooka/kaggleops-tutorial/blob/master/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

# 各種設定

In [ ]:
import os

try:
    from google.colab import auth
    auth.authenticate_user()
except ImportError:
    pass
    
REPOSITORY_NAME = 'kaggleops-tutorial'
PROJECT_ID = 'kaggleops-project-333212'
BUCKET_NAME = 'kaggleops-bucket-prd'

os.environ["GCLOUD_PROJECT"] = PROJECT_ID

In [ ]:
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [ ]:
!git clone https://github.com/supertrunks/{REPOSITORY_NAME}.git

In [ ]:
!pip install -qr ./{REPOSITORY_NAME}/requirements.txt

In [ ]:
cd ./{REPOSITORY_NAME}/src

In [ ]:
!gcloud config set project {PROJECT_ID}
!gcloud config configurations list

In [ ]:
# !gsutil -mq cp -rn gs://{BUCKET_NAME}/mlruns /content/{BUCKET_NAME}/src

blobs = storage_client.list_blobs(BUCKET_NAME)
for blob in blobs:
    filepath = blob.name
    if filepath[:7]=="mlruns/" and len(filepath)!=7:
        os.makedirs("/".join(filepath.split("/")[:-1]), exist_ok=True)
        blob.download_to_filename(filepath)

# プログラム実行

In [ ]:
!python mlproject.py

# ローカルに保存されたmlrunsをGCSに転送

In [ ]:
# !gsutil -mq cp -rn /content/{REPOSITORY_NAME}/src/mlruns gs://{BUCKET_NAME}

import glob

files = [f for f in glob.glob("mlruns/**", recursive=True) if f[:7]=="mlruns/" and len(f)!=7]
files_in_bucket = [f.name for f in storage_client.list_blobs(BUCKET_NAME)]
bucket = storage_client.get_bucket(BUCKET_NAME)

for f in files:
    try:
        if f not in files_in_bucket:
            blob = bucket.blob(f)
            blob.upload_from_filename(f)
    except:
        pass